In [1]:
#by Beatriz Domingues

In [53]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.support.ui import Select
sleep_sec = 8
sem_resultados=[]

### 1 . Carregar csv de Universidades

In [54]:
data = pd.read_csv('universidades.csv',';')
data.head()

,codigo,nome
0,53014014,ASSOCIAÇÃO DAS PIONEIRAS SOCIAIS (APS)
1,31102000,ASSOCIAÇÃO FÓRUM NACIONAL DE GESTORES DE INOVA...
2,25017012,ASSOCIAÇÃO INSTITUTO DE TECNOLOGIA DE PERNAMBU...
3,31008011,ASSOCIAÇÃO INSTITUTO NACIONAL DE MATEMÁTICA PU...
4,15024016,Associação Instituto Tecnológico Vale – Desenv...


### 2. Acessar o site

In [55]:
url = "https://sucupira.capes.gov.br/sucupira/public/consultas/coleta/envioColeta/dadosFotoEnvioColeta.jsf"
r = requests.get(url) 

soup = BeautifulSoup(r.content, 'html5lib') 
print(soup.prettify()) 


<!--?xml version="1.0" encoding="UTF-8"?-->
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="pt" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Plataforma Sucupira
  </title>
  <link href="/sucupira/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <meta content="UFRN - SINFO e CAPES - DTI" name="author"/>
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="sameorigin" http-equiv="X-Frame-Options"/>
  <meta content="nosniff" http-equiv="X-Content-Type-Options"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="max-age=0" http-equiv="cache-control"/>
  <meta content="no-cache" http-equiv="cache-control"/>
  <meta content="0" http-equiv="expires"/>
  <meta content="Tue, 01 Jan 1980 1:00:00 GMT" http-equiv="expires"/>
  <meta content="no-c

### 3. Abrir a página via Selenium

In [56]:
# create a new Chrome session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)


### 4. Inserir um codigo de instituição da busca

In [57]:
def set_codigoInstituicao(nome):
    try:
        search_institution = driver.find_element_by_id("form:j_idt30:inst:input")

        time.sleep(5)
        search_institution.clear()

        search_institution.send_keys(nome) 
        
        time.sleep(10)
    
    except NoSuchElementException:
        return False
    
    return True

### 6. Selecionar o primeiro valor do select de instituições


In [58]:
def get_nomesProgramas():
    programas = []
   
    i=1
    for select in driver.find_elements_by_css_selector('select'):
        value = select.get_attribute('name')
        if (i==3):
            j=0
            for tag in driver.find_elements_by_css_selector('select[name="'+value+'"] option'):
                value = tag.get_attribute('value')
                text = tag.text
                if (j>0):
                    programas.append(text)
                j+=1
        i+=1
    return programas 

In [59]:
def click_nome_instituicao():
    programas = []
   
    i=1
    for select in driver.find_elements_by_css_selector('select'):
        value = select.get_attribute('name')
        if (i==2):
            search_institution = driver.find_element_by_xpath("//select[@name='"+value+"']")
            search_institution.click()
        i+=1
        
    #return True


### 7. Pegar nomes dos programas

In [60]:
def get_nomesProgramas():
    programas = []
    i=1
    for select in driver.find_elements_by_css_selector('select'):
        value = select.get_attribute('name')
        if (i==3):
            j=0
            for tag in driver.find_elements_by_css_selector('select[name="'+value+'"] option'):
                value = tag.get_attribute('value')
                text = tag.text
                if (j>0):
                    programas.append(text)
                j+=1
        i+=1
    return programas 


### 8. Setar nome do programa

In [61]:
def set_nome_programa(nome):
    programas = []
    i=1
    for select in driver.find_elements_by_css_selector('select'):
        value = select.get_attribute('name')
        if (i==3):
            select.send_keys(nome)
        i+=1

### 9. CLicar no botão consultar


In [62]:
def click_botaoConsultar():
    try:
        botao_consultar = driver.find_element_by_id('form:consultar')
        time.sleep(sleep_sec)
        botao_consultar.click()
        time.sleep(sleep_sec)
    except NoSuchElementException:
        return False
    
    return True   

### 10. Expandir produções intelectuais

In [63]:
def click_producoes():
    try:
        teste = driver.find_element_by_xpath("//div[@id='producoes']/div")
        teste.click()
    except NoSuchElementException:
        return 0
    
    return 1 

In [64]:
#producoes_header = driver.find_element_by_id('form:j_idt92:j_idt312:cabecalhoProducoes')
#driver.find_element_by_xpath("//span[@id='form:j_idt92:j_idt312:cabecalhoProducoes']/a").click()
#driver.find_element_by_xpath("//span[@id='form:j_idt92:j_idt312:cabecalhoProducoes']/a").click()
#producoes_header.click()

### 11. Guardar a div de produções intelectuais

In [65]:
def get_contentProducoes():
    try:
        conteudo_producoes = driver.find_element_by_xpath("//div[@id='producoes']/div[@class='panel-collapse collapse in']")
    except NoSuchElementException:
        return 0
    
    return conteudo_producoes 

### 12. pegar o link da div de producoes intelectuais (XLS)

In [66]:
def get_linkXLS(content_producoes):
    try:
        links_xls = content_producoes.find_elements_by_partial_link_text('XLS')
        time.sleep(10)
        link_para_clicar = "";
        for l in links_xls:
            link_para_clicar = l
    except NoSuchElementException:
        return 0
    
    return link_para_clicar 

### 14. Baixar relatorios de todos os programas do INPE


In [67]:
 df_instituicoes = data[415:429]

In [68]:
df_instituicoes.head()

,codigo,nome
415,40005011,UNIVERSIDADE ESTADUAL DE PONTA GROSSA (UEPG)
416,13003011,UNIVERSIDADE ESTADUAL DE RORAIMA (UERR)
417,28007018,UNIVERSIDADE ESTADUAL DE SANTA CRUZ (UESC)
418,22003010,UNIVERSIDADE ESTADUAL DO CEARÁ (UECE)
419,40014010,UNIVERSIDADE ESTADUAL DO CENTRO-OESTE (UNICENTRO)


In [69]:
df_instituicoes = df_instituicoes['codigo']
df_instituicoes

415    40005011
416    13003011
417    28007018
418    22003010
419    40014010
420    20002017
421    40031012
422    31033016
423    40015017
424    40076016
425    42045010
426    28006011
427    16001010
428    35017007
Name: codigo, dtype: int64

In [70]:
com_resultados = []
sem_resultados = []
    
for i in df_instituicoes:

    driver.execute_script("location.reload()")

    set_nome_instituicao_res = set_codigoInstituicao(i)

    time.sleep(15)

    click_nome_instituicao()

    time.sleep(10)

    programas = get_nomesProgramas()
    #print(programas)

   
    if (len(programas)>0):
         for p in programas:
            set_nome_programa(p)
            time.sleep(10)
            click_botaoConsultar()
            time.sleep(10)
            click_producoes()
            time.sleep(10)
            content_producoes = get_contentProducoes()
            #print(content_producoes)
            time.sleep(10)
            if (content_producoes!=0):
                link = get_linkXLS(content_producoes)
                if (link!=0):
                    com_resultados.append(str(i)+" - "+ p)
                    link.click()
            else:
                sem_resultados.append(str(i)+" - "+ p)
    else:
        sem_resultados.append(i)
    
    print(sem_resultados)

['40005011 - EDUCAÇÃO INCLUSIVA (33004129069P9) (Programa em Rede) ']
['40005011 - EDUCAÇÃO INCLUSIVA (33004129069P9) (Programa em Rede) ']
['40005011 - EDUCAÇÃO INCLUSIVA (33004129069P9) (Programa em Rede) ', '28007018 - CULTURA & TURISMO - UESC - UFBA (28007018002P0)', '28007018 - ENFERMAGEM (28007018077P0)', '28007018 - HISTÓRIA: ATLÂNTICO E DIÁSPORA AFRICANA (28007018076P3)']
['40005011 - EDUCAÇÃO INCLUSIVA (33004129069P9) (Programa em Rede) ', '28007018 - CULTURA & TURISMO - UESC - UFBA (28007018002P0)', '28007018 - ENFERMAGEM (28007018077P0)', '28007018 - HISTÓRIA: ATLÂNTICO E DIÁSPORA AFRICANA (28007018076P3)', '22003010 - ADMINISTRAÇÃO (22003010002P8)', '22003010 - HISTÓRIA E CULTURAS (22003010020P6)', '22003010 - SAÚDE COLETIVA (22003010021P2) (Programa em Rede) ']
['40005011 - EDUCAÇÃO INCLUSIVA (33004129069P9) (Programa em Rede) ', '28007018 - CULTURA & TURISMO - UESC - UFBA (28007018002P0)', '28007018 - ENFERMAGEM (28007018077P0)', '28007018 - HISTÓRIA: ATLÂNTICO E DIÁSPORA

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=77.0.3865.90)


In [71]:
sem_resultados

['40005011 - EDUCAÇÃO INCLUSIVA (33004129069P9) (Programa em Rede) ',
 '28007018 - CULTURA & TURISMO - UESC - UFBA (28007018002P0)',
 '28007018 - ENFERMAGEM (28007018077P0)',
 '28007018 - HISTÓRIA: ATLÂNTICO E DIÁSPORA AFRICANA (28007018076P3)',
 '22003010 - ADMINISTRAÇÃO (22003010002P8)',
 '22003010 - HISTÓRIA E CULTURAS (22003010020P6)',
 '22003010 - SAÚDE COLETIVA (22003010021P2) (Programa em Rede) ',
 '40014010 - NANOCIÊNCIAS E BIOCIÊNCIAS (40014010043P5)',
 '20002017 - EDUCAÇÃO (20002017042P8)',
 '20002017 - EDUCAÇÃO INCLUSIVA (33004129069P9) (Programa em Rede) ',
 '20002017 - ENGENHARIA AEROESPACIAL (23001011181P0) (Programa em Rede) ',
 '40031012 - EDUCAÇÃO (40031012071P3)',
 '40015017 - CIÊNCIA DA COMPUTAÇÃO (40015017173P2)',
 '40015017 - ENGENHARIA E TECNOLOGIA AMBIENTAL  (40001016173P5) (Programa em Rede) ',
 '40015017 - História (40015017007P5)',
 '40015017 - LETRAS (23001011069P5) (Programa em Rede) ',
 '40015017 - Letras (40015017003P0)',
 '40015017 - Matemática em Rede Na

In [ ]:
if (link!=0):
            com_resultados.append(str(i)+" - "+ p)

In [72]:
com_resultados

['40005011 - AGRONOMIA (40005011006P7)',
 '40005011 - Bioenergia - UEL - UEM - UEPG - UNICENTRO - UNIOESTE - UFPR (40002012041P8) (Programa em Rede) ',
 '40005011 - BIOLOGIA EVOLUTIVA (40005011011P0) (Programa em Rede) ',
 '40005011 - CIÊNCIA E TECNOLOGIA DE ALIMENTOS (40005011007P3)',
 '40005011 - CIÊNCIAS (40005011008P0)',
 '40005011 - CIÊNCIAS BIOMÉDICAS (40005011018P5)',
 '40005011 - CIÊNCIAS DA SAÚDE (40005011170P1)',
 '40005011 - CIÊNCIAS FARMACÊUTICAS (40014010006P2) (Programa em Rede) ',
 '40005011 - CIÊNCIAS SOCIAIS APLICADAS (40005011003P8)',
 '40005011 - Computação Aplicada (40005011013P3)',
 '40005011 - ECONOMIA (40005011171P8)',
 '40005011 - EDUCAÇÃO (40005011005P0)',
 '40005011 - ENGENHARIA E CIÊNCIA DE MATERIAIS (40005011002P1)',
 '40005011 - Engenharia Sanitária e Ambiental - UNICENTRO (40005011017P9) (Programa em Rede) ',
 '40005011 - ENSINO DE CIÊNCIAS E EDUCAÇÃO MATEMÁTICA (40005011172P4)',
 '40005011 - Ensino de Física  - PROFIS (33283010001P5) (Programa em Rede) ',

In [27]:
data[410:430]

,codigo,nome
410,40002012,UNIVERSIDADE ESTADUAL DE LONDRINA (UEL)
411,40004015,UNIVERSIDADE ESTADUAL DE MARINGÁ (UEM)
412,51004011,UNIVERSIDADE ESTADUAL DE MATO GROSSO DO SUL (U...
413,51004020,"UNIVERSIDADE ESTADUAL DE MATO GROSSO DO SUL ""D..."
414,32014015,UNIVERSIDADE ESTADUAL DE MONTES CLAROS (UNIMON...
415,40005011,UNIVERSIDADE ESTADUAL DE PONTA GROSSA (UEPG)
416,13003011,UNIVERSIDADE ESTADUAL DE RORAIMA (UERR)
417,28007018,UNIVERSIDADE ESTADUAL DE SANTA CRUZ (UESC)
418,22003010,UNIVERSIDADE ESTADUAL DO CEARÁ (UECE)
419,40014010,UNIVERSIDADE ESTADUAL DO CENTRO-OESTE (UNICENTRO)
